In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold,KFold,GroupKFold
import xgboost as xgb
from datetime import datetime
from kaggle.competitions import nflrush
import math
import tqdm
from scipy.special import expit
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d, ConvexHull
pd.set_option("display.max_rows",1000)
env = nflrush.make_env()

In [2]:
def crps_score(y_pred, data):
    y_pred = y_pred.reshape(-1,199)
    y_true = np.array(data.get_label()).astype("int")
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return "CRPS", ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

def simple_crps(y_true, y_pred):
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

In [3]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',low_memory=False)

# preprocess and feature engineering 

In [4]:
def transform_time_quarter(str1):
    return int(str1[:2])*60 + int(str1[3:5])
  
def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
      
def back_direction(orientation):
    if orientation > 180.0:
        return 1
    else:
        return 0
    
def transform_height(te):
    return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100

def voronoi_volumes(points, selected_index):
    v = Voronoi(points)
    vol = np.zeros(v.npoints)
      
    for i, reg_num in enumerate(v.point_region):
        if reg_num == v.point_region[selected_index]:
            indices = v.regions[reg_num]
            if -1 in indices: # some regions can be opened
                vol = -999 ## insert missing value when the area is open
            else:
                vol = ConvexHull(v.vertices[indices]).volume      
            break
    return vol

def radius_calc(dist_to_ball):
    return 4 + 6 * (dist_to_ball >= 15) + (dist_to_ball ** 3) / 560 * (dist_to_ball < 15)

def compute_influence_to_ball(row):
    point = np.array([row["RushX"], row["RushY"]])
    theta = math.radians(row['Orientation'])
    speed = row['S']
    player_coords = row[['X', 'Y']].values
    dist_to_ball = row["DisToRusher"]    

    S_ratio = (speed / 13) ** 2    # we set max_speed to 13 m/s ((dominance_df["S"]*0.9144).max() #8.64m/s)
    RADIUS = radius_calc(dist_to_ball)  # updated

    S_matrix = np.matrix([[RADIUS * (1 + S_ratio), 0], [0, RADIUS * (1 - S_ratio)]])
    R_matrix = np.matrix([[np.cos(theta), - np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    COV_matrix = np.dot(np.dot(np.dot(R_matrix, S_matrix), S_matrix), np.linalg.inv(R_matrix))
    
    norm_fact = (1 / 2 * np.pi) * (1 / np.sqrt(np.linalg.det(COV_matrix)))    
    mu_play = player_coords + speed * np.array([np.cos(theta), np.sin(theta)]) / 2
    
    intermed_scalar_player = np.dot(np.dot((player_coords - mu_play),
                                    np.linalg.inv(COV_matrix)),
                             np.transpose((player_coords - mu_play)))
    player_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_player[0, 0])
    
    intermed_scalar_point = np.dot(np.dot((point - mu_play), 
                                    np.linalg.inv(COV_matrix)), 
                             np.transpose((point - mu_play)))
    point_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_point[0, 0])
    
    return point_influence / player_influence

In [5]:
remove_features = ['GameId','PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap', 'PlayDirection', 'TeamOnOffense', 
                    'PlayerBirthDate', 'is_run', 'NflIdRusher', 'date_game', 'RushX', 'RushY', 'PossessionTeam', 
                   'FieldPosition', 'Position', 'PlayerHeight', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Turf', 'Quarter', 'OffenseFormation'
                  , 'RushX_0.5s', 'RushY_0.5s', 'RushX_1s', 'RushY_1s']
top20_weather = list(train.GameWeather.value_counts(normalize=True, dropna=False).cumsum().head(20).index)
top20_winddirection = list(train.WindDirection.value_counts(normalize=True, dropna=False).cumsum().head(20).index)

In [6]:
def transform_data(df):
    df.loc[df.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

    df['is_run'] = df.NflId == df.NflIdRusher

    df['S'] = 10 * df['Dis']
    df.loc[df.Season == 2017, 'A'] = df[df.Season == 2018]['A'].mean()
    
    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['OnOffense'] = df.Team == df.TeamOnOffense 
    df['YardLine_std'] = 100 - df.YardLine.copy()
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
            'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X.copy()
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y.copy()
    df.loc[df.ToLeft, 'Y_std'] = 53.3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = df.Orientation.copy()
    df.loc[df.ToLeft, 'Orientation_std'] = np.mod(180 + df.loc[df.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir.copy()
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(180 + df.loc[df.ToLeft, 'Dir_std'], 360)
    df.loc[df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + df.loc[df['Season'] == 2017, 'Orientation_std'], 360) 
    df.drop(["X", "Y", "Orientation", "YardLine", "Dir", "ToLeft"], axis=1, inplace=True)
    df.rename(columns={'X_std': 'X', 'Y_std': 'Y', 'Orientation_std': 'Orientation', 'Dir_std': 'Dir', "YardLine_std": "YardLine"}, inplace=True)
    
    df['date_game'] = df.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    df['age'] = (df.date_game.map(pd.to_datetime) - df.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365

    df["Momentum"] = df["S"] * df["PlayerWeight"]
    
    df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    
    df["Momentum_cos"] = df["Momentum"] * df["Dir_cos"]
    
    df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin((450-x) * np.pi/ 180))
    
    df["S_cos"] = df["S"] * df["Dir_cos"]
    df["S_sin"] = df["S"] * df["Dir_sin"]
    df["X_0.5s"] = df["X"] + df["S_cos"] * 0.5
    df["Y_0.5s"] = df["Y"] + df["S_sin"] * 0.5
    df["X_1s"] = df["X"] + df["S_cos"] * 1
    df["Y_1s"] = df["Y"] + df["S_sin"] * 1

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X"].agg(np.mean)[1::2])
    df["RushX"] = np.repeat(rusher_x, 22) # repeat each elemnt 22 times
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y"].agg(np.mean)[1::2])
    df["RushY"] = np.repeat(rusher_y, 22) 
    df["DisToRusher"] = np.sqrt((df["X"] - df["RushX"]) ** 2 + (df["Y"] - df["RushY"]) ** 2)
    df["TackleTimeToRusher"] = df["DisToRusher"] / df["S"] # includes nan when the speed of rusher is 0
    
    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_0.5s"].agg(np.mean)[1::2])
    df["RushX_0.5s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_0.5s"].agg(np.mean)[1::2])
    df["RushY_0.5s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_0.5s"] = np.sqrt((df["X_0.5s"] - df["RushX_0.5s"]) ** 2 + (df["Y_0.5s"] - df["RushY_0.5s"]) ** 2)
    df["TackleTimeToRusher_0.5s"] = df["DisToRusher_0.5s"] / df["S"]

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_1s"].agg(np.mean)[1::2])
    df["RushX_1s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_1s"].agg(np.mean)[1::2])
    df["RushY_1s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_1s"] = np.sqrt((df["X_1s"] - df["RushX_1s"]) ** 2 + (df["Y_1s"] - df["RushY_1s"]) ** 2)
    df["TackleTimeToRusher_1s"] = df["DisToRusher_1s"] / df["S"]
    
    #df["X_0.5s"] = df["X"] + (df["S_cos"] * 0.5 + 0.5 * df["A"] * df["Dir_cos"] * (0.5**2) )
    #df["Y_0.5s"] = df["Y"] + (df["S_sin"] * 0.5 + 0.5 * df["A"] * df["Dir_sin"] * (0.5**2) )
    #df["X_1s"] = df["X"] + (df["S_cos"] * 1 + 0.5 * df["A"] * df["Dir_cos"] * (1**2) ) 
    #df["Y_1s"] = df["Y"] + (df["S_sin"] * 1 + 0.5 * df["A"] * df["Dir_sin"] * (1**2) ) 
    #df["X_2s"] = df["X"] + (df["S_cos"] * 2 + 0.5 * df["A"] * df["Dir_cos"] * (2**2) ) 
    #df["Y_2s"] = df["Y"] + (df["S_sin"] * 2 + 0.5 * df["A"] * df["Dir_sin"] * (2**2) )
    
    df["AttackAngle"] =  np.arctan((df["Y"] - df["RushY"]) / (df["X"] - df["RushX"])) * 180 / np.pi
    
    df_single = df[df.is_run==True].copy()
        
    df_single['time_end'] = df_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)

    df_single["Stadium"] = df_single["Stadium"].map(lambda x: "Broncos Stadium at Mile High" if x=="Broncos Stadium At Mile High" 
                                             else ("CenturyLink Field" if x == "CenturyField" or x == x=="CenturyLink"
                                             else ("Everbank Field" if x == "EverBank Field"
                                             else ("FirstEnergy Stadium" if x =="First Energy Stadium" or x=="FirstEnergy" or x == "FirstEnergyStadium"
                                             else ("Lambeau Field" if x == "Lambeau field"
                                             else ("Los Angeles Memorial Coliseum" if x == "Los Angeles Memorial Coliesum"
                                             else ("M&T Bank Stadium" if x == "M & T Bank Stadium" or x == "M&T Stadium"
                                             else ("Mercedes-Benz Superdome" if x == "Mercedes-Benz Dome"
                                             else ("MetLife Stadium" if x == "MetLife" or x == "Metlife Stadium"
                                             else ("NRG Stadium" if x == "NRG"
                                             else ("Oakland-Alameda County Coliseum" if x == "Oakland Alameda-County Coliseum"
                                             else ("Paul Brown Stadium" if x == "Paul Brown Stdium"
                                             else ("Twickenham Stadium" if x == "Twickenham" else x)))))))))))))

    df_single["Location"] = df_single["Location"].map(lambda x: "Arlington, TX" if x == "Arlington, Texas"
                                            else ("Baltimore, MD" if x == "Baltimore, Maryland" or x == "Baltimore, Md."
                                            else ("Charlotte, NC" if x == "Charlotte, North Carolina"
                                            else ("Chicago, IL" if x == "Chicago. IL"
                                            else ("Cincinnati, OH" if x == "Cincinnati, Ohio"
                                            else ("Cleveland, OH" if x == "Cleveland" or x == "Cleveland Ohio" or x == "Cleveland, Ohio" or x == "Cleveland,Ohio"
                                            else ("Detroit, MI" if x == "Detroit"
                                            else ("East Rutherford, NJ" if x == "E. Rutherford, NJ" or x == "East Rutherford, N.J."
                                            else ("Foxborough, MA" if x == "Foxborough, Ma"
                                            else ("Houston, TX" if x == "Houston, Texas"
                                            else ("Jacksonville, FL" if x == "Jacksonville Florida" or x == "Jacksonville, Fl" or x == "Jacksonville, Florida"
                                            else ("London" if x == "London, England"
                                            else ("Los Angeles, CA" if x == "Los Angeles, Calif."
                                            else ("Miami Gardens, FLA" if x == "Miami Gardens, Fla."
                                            else ("New Orleans, LA" if x == "New Orleans" or x == "New Orleans, La."
                                            else ("Orchard Park, NY" if x == "Orchard Park NY"
                                            else ("Philadelphia, PA" if x == "Philadelphia, Pa."
                                            else ("Pittsburgh, PA" if x == "Pittsburgh"
                                            else ("Seattle, WA" if x == "Seattle" else x)))))))))))))))))))

    grass_labels = ['grass', 'natural grass', 'natural', 'naturall grass']
    df_single['Grass'] = np.where(df_single.Turf.str.lower().isin(grass_labels), "Natural", "Artificial")
                                                                                                                             
    df_single['DefendersInTheBox_vs_Distance'] = df_single['DefendersInTheBox'] / df_single['Distance']
                                                                   
    df_single["DisToQB"] = np.array(df[(df.Position=="QB") | (df.Position=="C")].groupby(["PlayId"]).agg(np.mean)["DisToRusher"])

    df_single['runner_height'] = df_single.PlayerHeight.map(transform_height)
    df_single.drop(remove_features,axis=1,inplace=True) 

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseAveX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseAveX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseAveY"] = np.array(tmp[0::2]["Y"]) 
    df_single["OffenseAveY"] = np.array(tmp[1::2]["Y"]) 
    df_single["DefenseAveMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseAveMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseAveMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseAveMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    #tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X_0.5s", "Y_0.5s", "X_1s", "Y_1s"]]
    #df_single["DefenseAveX_0.5s"] = np.array(tmp[0::2]["X_0.5s"])
    #df_single["OffenseAveX_0.5s"] = np.array(tmp[1::2]["X_0.5s"])
    #df_single["DefenseAveY_0.5s"] = np.array(tmp[0::2]["Y_0.5s"]) 
    #df_single["OffenseAveY_0.5s"] = np.array(tmp[1::2]["Y_0.5s"]) 
    #df_single["DefenseAveX_1s"] = np.array(tmp[0::2]["X_1s"])
    #df_single["OffenseAveX_1s"] = np.array(tmp[1::2]["X_1s"])
    #df_single["DefenseAveY_1s"] = np.array(tmp[0::2]["Y_1s"]) 
    #df_single["OffenseAveY_1s"] = np.array(tmp[1::2]["Y_1s"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseStdX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseStdX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseStdY"] = np.array(tmp[0::2]["Y"])
    df_single["OffenseStdY"] = np.array(tmp[1::2]["Y"])
    df_single["DefenseStdMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseStdMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseStdMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseStdMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X_0.5s", "Y_0.5s", "X_1s", "Y_1s"]]
    df_single["DefenseStdX_0.5s"] = np.array(tmp[0::2]["X_0.5s"])
    df_single["OffenseStdX_0.5s"] = np.array(tmp[1::2]["X_0.5s"])
    df_single["DefenseStdY_0.5s"] = np.array(tmp[0::2]["Y_0.5s"]) 
    df_single["OffenseStdY_0.5s"] = np.array(tmp[1::2]["Y_0.5s"]) 
    df_single["DefenseStdX_1s"] = np.array(tmp[0::2]["X_1s"])
    df_single["OffenseStdX_1s"] = np.array(tmp[1::2]["X_1s"])
    df_single["DefenseStdY_1s"] = np.array(tmp[0::2]["Y_1s"]) 
    df_single["OffenseStdY_1s"] = np.array(tmp[1::2]["Y_1s"])
    
    df_single["RunnerToDefenseCentoid"] = np.sqrt((df_single["X"] - df_single["DefenseAveX"]) ** 2 + (df_single["Y"] - df_single["DefenseAveY"]) ** 2)
    df_single["RunnerToOffenseCentoid"] = np.sqrt((df_single["X"] - df_single["OffenseAveX"]) ** 2 + (df_single["Y"] - df_single["OffenseAveY"]) ** 2)

    tmp_max = df.groupby(["PlayId", "OnOffense"])["X"].max()
    tmp_min = df.groupby(["PlayId", "OnOffense"])["X"].min()
    df_single["DefenseSpreadX"] = np.array(tmp_max[0::2]- tmp_min[0::2])
    df_single["OffenseSpreadX"] = np.array(tmp_max[1::2]- tmp_min[1::2])

    df_single["RunnerToScrimmage"] = df_single["X"] - (df_single["YardLine"] + 10)

    df_single["MinTackleTime"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher"].min()[0::2])
    df_single["MinTackleTime_0.5s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_0.5s"].min()[0::2])
    df_single["MinTackleTime_1s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_1s"].min()[0::2])
    df_single["1stDefender_Momentum_cos"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_cos"])
    first_offenser_index =df.groupby(["PlayId", "OnOffense"])["DisToRusher"].nsmallest(2)[3::4].reset_index()["level_2"]
    df_single["1stOffenser_Momentum_cos"] = np.array(df.loc[first_offenser_index]["Momentum_cos"])
    df_single["1stDefender_AttackAngle"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["AttackAngle"])

    pts = np.array(df[["X", "Y"]]).reshape(df.shape[0]//22, 22, 2) # plays * players * (X, Y, rusher)
    rusher_index = list(df[df.is_run==True].index % 22) 
    closest_def_index = list(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]].index % 22)
    rusher_voronoi = []
    closest_def_voronoi = []

    for i in range(0, df.shape[0] //22):
        rusher_voronoi.append(voronoi_volumes(pts[i], rusher_index[i]))
        closest_def_voronoi.append(voronoi_volumes(pts[i], closest_def_index[i]))
    df_single["RusherVoronoi"] = rusher_voronoi    
    df_single["FirstDefenderVoronoi"] = closest_def_voronoi 
        
    df_single.fillna(-999,inplace=True) 
    remove_features2 = ["OnOffense", "DisToRusher", "TackleTimeToRusher", "OffenseAveX", "OffenseAveY", "AttackAngle", "Dis",
                       "DisToRusher_0.5s", "TackleTimeToRusher_0.5s", "DisToRusher_1s", "TackleTimeToRusher_1s"]
    df_single.drop(remove_features2, axis=1, inplace=True)

    return df_single

In [7]:
%%time
train_single = transform_data(train)
y_train = train_single.Yards + 99 # to categorize
X_train = train_single.drop(['Yards'],axis=1)
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))

/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in arctan
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: user 5min 52s, sys: 4.66 s, total: 5min 57s
Wall time: 5min 53s


# modelling

In [8]:
# modified CV
df_20172018 = pd.concat([X_train, y_train], axis=1)
df_2017 = df_20172018[df_20172018.Season==2017].copy()
df_2018 = df_20172018[df_20172018.Season==2018].copy()
X_2017 = df_2017.drop(['Yards'],axis=1)
y_2017 = df_2017.Yards
X_2018 = df_2018.drop(['Yards'],axis=1)
y_2018 = df_2018.Yards
groups = np.array(X_2018.Week)
n_folds= 3
gkf=GroupKFold(n_splits = n_folds)
average_crps = 0
models = []
crps_scores = []
xgb_params = {
    "objective" : "multi:softprob",
    "eval_metric" : "mlogloss", 
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_class": 199,
    "num_leaves" : 13,
    "learning_rate" : 0.05,
}
evals_result = {}
num_boost_round=100000
# 2017 data: training only, 2018 data: separate by cross validation
for i , (train_index, test_index) in enumerate(gkf.split(X_2018, y_2018, groups)):
    X_train_2018 = X_2018.iloc[train_index,:]
    y_train_2018 = y_2018.iloc[train_index]
    X_test2 = X_2018.iloc[test_index,:]
    y_test2 = y_2018.iloc[test_index]
    X_train2 = pd.concat([X_2017, X_train_2018])
    y_train2 = pd.concat([y_2017, y_train_2018])
    xgb_train = xgb.DMatrix(X_train2, label = y_train2)
    xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
    watchlist = [(xgb_eval, "eval")]
    
    clf = xgb.train(
        xgb_params, xgb_train, num_boost_round, watchlist,
        early_stopping_rounds=10,
        evals_result=evals_result,
        feval=crps_score,
    )
    
    models.append(clf)
    temp_predict = clf.predict(xgb_eval, ntree_limit=clf.best_ntree_limit)
    score = simple_crps(y_test2, temp_predict)
    print(score)
    crps_scores.append(score)
    average_crps += score / n_folds
    if i == 0:
        feature_importance_df = pd.DataFrame(clf.get_score(importance_type="total_gain").items(), columns =["Features", "Fold_"+str(i+1)])
    else:
        feature_importance_df["Fold_"+str(i+1)] = list(clf.get_score(importance_type="total_gain").values())
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
print('crps list:', crps_scores)
print('average crps:',average_crps)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	eval-mlogloss:4.89166	eval-CRPS:0.08062
Multiple eval metrics have been passed: 'eval-CRPS' will be used for early stopping.

Will train until eval-CRPS hasn't improved in 10 rounds.
[1]	eval-mlogloss:4.66709	eval-CRPS:0.076978
[2]	eval-mlogloss:4.50037	eval-CRPS:0.073636
[3]	eval-mlogloss:4.366	eval-CRPS:0.070526
[4]	eval-mlogloss:4.25277	eval-CRPS:0.067598
[5]	eval-mlogloss:4.15722	eval-CRPS:0.064857
[6]	eval-mlogloss:4.07333	eval-CRPS:0.062258
[7]	eval-mlogloss:3.99914	eval-CRPS:0.059803
[8]	eval-mlogloss:3.93242	eval-CRPS:0.057461
[9]	eval-mlogloss:3.87172	eval-CRPS:0.055251
[10]	eval-mlogloss:3.8168	eval-CRPS:0.05316
[11]	eval-mlogloss:3.76652	eval-CRPS:0.051173
[12]	eval-mlogloss:3.72006	eval-CRPS:0.049295
[13]	eval-mlogloss:3.67781	eval-CRPS:0.047507
[14]	eval-mlogloss:3.63817	eval-CRPS:0.045808
[15]	eval-mlogloss:3.60118	eval-CRPS:0.044194
[16]	eval-mlogloss:3.56644	eval-CRPS:0.042664
[17]	eval-mlogloss:3.53418	eval-CRPS:0.041211
[18]	eval-mlogloss:3.50408	eval-CRPS:0.03983

- add std of X_0.5s ~ Y_1.0s by offense and defense
- crps list: [0.013769273527182612, 0.013754405269255355, 0.013694276133494061]
- average crps: 0.013739318309977343
- 
- adding mintackletime0.5s, mintackletime1s
- crps list: [0.013534292471748168, 0.013498420075932873, 0.013412049215626116]
- average crps: 0.013481587254435719

In [9]:
X_train.columns

Index(['Team', 'S', 'A', 'NflId', 'JerseyNumber', 'Season', 'Down', 'Distance',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel', 'PlayerWeight',
       'PlayerCollegeName', 'Week', 'Stadium', 'Location', 'StadiumType',
       'GameWeather', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection',
       'YardLine', 'X', 'Y', 'Orientation', 'Dir', 'age', 'Momentum',
       'Dir_cos', 'Momentum_cos', 'Dir_sin', 'S_cos', 'S_sin', 'X_0.5s',
       'Y_0.5s', 'X_1s', 'Y_1s', 'time_end', 'Grass',
       'DefendersInTheBox_vs_Distance', 'DisToQB', 'runner_height',
       'DefenseAveX', 'DefenseAveY', 'DefenseAveMomentum',
       'OffenseAveMomentum', 'DefenseAveMomentum_cos',
       'OffenseAveMomentum_cos', 'DefenseStdX', 'OffenseStdX', 'DefenseStdY',
       'OffenseStdY', 'DefenseStdMomentum', 'OffenseStdMomentum',
       'DefenseStdMomentum_cos', 'OffenseStdMomentum_cos', 'DefenseStdX_0.5s',
       'OffenseStdX_0.5s'

In [10]:
feature_importance_df.sort_values("Average").head(80).reset_index(drop=True)

,Features,Fold_1,Fold_2,Fold_3,Average,Std,Cv
0,Season,217.462711,261.275693,393.959563,290.899322,75.037565,0.257950
1,Grass,475.393560,423.745898,279.007774,392.715744,83.122388,0.211660
2,Down,586.359357,1314.718104,1003.298919,968.125460,298.389561,0.308214
3,Team,346.420294,2855.518442,634.828389,1278.922375,1121.022205,0.876537
4,DefendersInTheBox,1453.434120,2507.813814,194.864124,1385.370686,945.483488,0.682477
5,DefensePersonnel,1071.103908,1392.561877,2479.444119,1647.703301,602.593504,0.365717
6,RusherVoronoi,2364.701688,1833.648192,1097.177029,1765.175636,519.724965,0.294432
7,Y_1s,2504.456261,778.981749,2604.990809,1962.809606,838.098282,0.426989
8,Y_0.5s,2504.173879,1276.981814,3046.371333,2275.842342,740.173982,0.325231
9,PlayerCollegeName,4020.402279,1358.084740,1520.376193,2299.621071,1218.578572,0.529904


# prediction

In [11]:
for (test_df, sample_prediction_df) in env.iter_test():
    X_test = transform_data(test_df)
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    dtest = xgb.DMatrix(X_test)
    for model in models:
        pred_value += model.predict(dtest, ntree_limit = model.best_ntree_limit)[0] / n_folds
    pred_data = np.clip(np.cumsum(pred_value), 0, 1)
    pred_data = np.array(pred_data).reshape(1,199)
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               data = pred_data)
    env.predict(pred_target)
env.write_submission_file()

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
